In [54]:
"""
https://leetcode.com/problems/design-search-autocomplete-system/
subscription needed

https://aaronice.gitbook.io/lintcode/data_structure/design-search-autocomplete-system

Design a search autocomplete system for a search engine.
Users may input a sentence (at least one word and end with a special character'#').
Foreach character they type except '#', you need to return the top 3 historical hot sentences
that have prefix the same as the part of sentence already typed. Here are the specific rules:

1. The hot degree for a sentence is defined as the number of times a user typed the exactly same sentence before.
2. The returned top 3 hot sentences should be sorted by hot degree (The first is the hottest one).
   If several sentences have the same degree of hot, you need to use ASCII-code order (smaller one appears first).
3. If less than 3 hot sentences exist, then just return as many as you can.
4. When the input is a special character, it means the sentence ends, and in this case, you need to return an empty list.

Your job is to implement the following functions:

AutocompleteSystem(String[] sentences, int[] times):
   This is the constructor.
   The input ishistorical data.
   Sentencesis a string array consists of previously typed sentences.
   Times is the corresponding times a sentence has been typed. Your system should record these historical data.
   
Now, the user wants to input a new sentence.
The following function will provide the next character the user types:

List<String> input(char c):
   The inputcis the next character typed by the user.
   The character will only be lower-case letters ('a'to'z'), blank space (' ') or a special character ('#').
   Also, the previously typed sentence should be recorded in your system.
   The output will be thetop 3historical hot sentences that have prefix the same as the part of sentence already typed.


Note:
1. The input sentence will always start with a letter and end with '#', and only one blank space will exist between two words.
2. The number of complete sentences that to be searched won't exceed 100.
   The length of each sentence including those in the historical data won't exceed 100.
3. Please use double-quote instead of single-quote when you write test cases even for a character input.
4. Please remember to RESET your class variables declared in class AutocompleteSystem, as static/class variables are
   persisted across multiple test cases . Please see here for more details.
"""

# we can use a character Trie
# 
#       #root# 
#         i
#   ' '   r   s 
#   l     o   l
#   o     n   a
#   v     m   n
#   e     a   d
#  ' '    n   #
#  y l    #
#  o e
#  u e
#  # t
#    c
#    o
#    d
#    e
#    #


from collections import defaultdict

class Node:
    def __init__(self):
        self.children = {}
        self.sentences = defaultdict(int)

    def print(self):
        print(f'{self.children = }, {self.sentences =}')        
        for k,v in self.children.items():
            print(f'{k = }')
            v.print()
    
class AutocompleteSystem():
    def __init__(self, sentences, freqs):
        self.trieroot = Node()
        for (sen,freq) in zip(sentences, freqs):
            self._add_to_trie(sen,freq)
        # self.trieroot.print()
        
        self.userstate = self.trieroot
        self.userinput = ''

    def _add_to_trie(self, sen, freq):
        n = self.trieroot
        for c in sen:
            if c not in n.children:
                n.children[c]=Node()
            n = n.children[c]
            n.sentences[sen] += freq
    
    def input(self, c):
        if c == '#':
            # add userinput
            self._add_to_trie(self.userinput, 1)
            # reset state
            self.userstate = self.trieroot
            self.userinput = ''
            return []

        self.userinput += c

        if self.userstate == None:
            return []

        if c not in self.userstate.children:
            self.userstate = None
            return []
            
        self.userstate = self.userstate.children[c]
        # get top 3 from self.userstate.sen
        sortedSens = sorted([(-f,s) for (s,f) in self.userstate.sentences.items()])
        retVal = [s for _, s in sortedSens[:3]]
        return retVal


autocomplete = AutocompleteSystem(["i love you", "island", "ironman", "i love leetcode"], [5,3,2,2])
# "i love you": 5 times
# "island": 3 times
# "ironman": 2 times
# "i love leetcode": 2 times


assert(autocomplete.input('i') == ["i love you", "island", "i love leetcode"])
# There are four sentences that have prefix "i".
# Among them, "ironman" and "i love leetcode" have same hot degree.
# Since' ' has ASCII code 32 and 'r' has ASCII code 114, "i love leetcode" should be in front of "ironman".
# Also we only need to output top 3 hot sentences, so "ironman" will be ignored.

assert(autocomplete.input(' ') ==["i love you", "i love leetcode"])
# There are only two sentences that have prefix "i".

assert(autocomplete.input('a') == [])
# There are no sentences that have prefix"i a".

assert(autocomplete.input('#') == [])
# The user finished the input
# the sentence "i a" should be saved as a historical sentence in system.
# And the following input will be counted as a new search.
